## This notebook check the hexapod state transitions and move/offset commands
## It also checks the LUT against the input polynomials, before and after a slew

This notebook works with both hexapods.

To switch between the hexapods, change the cell below.

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

use_ptg = 1 #if using ptg+mount for telemetry change
#use_ptg = 0

In [ ]:
from lsst.ts import salobj
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts.idl.enums import MTHexapod

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

In [ ]:
if summit:
    print(os.environ["OSPL_URI"])
    print(os.environ["LSST_DDS_PARTITION_PREFIX"])
    print(os.environ["LSST_DDS_DOMAIN_ID"])
else:
    print(os.environ["OSPL_URI"])
    if os.environ.get("LSST_DDS_ALIGNER", "false") != "false":
        print("LSST_DDS_ALIGNER is mis-configured")

In [ ]:
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1

In [ ]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7)

In [ ]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components_attr

In [ ]:
#depending on which hexapod we are testing, comment/uncomment below accordingly
hexa = mtcs.rem.mthexapod_1  #camera hexapod
#hexa = mtcs.rem.mthexapod_2 #M2 hexapod 

ptg = mtcs.rem.mtptg
mount = mtcs.rem.mtmount
rot = mtcs.rem.mtrotator

In [ ]:
#As long as you get something its OK. we dont' care about h.heartbeat
await hexa.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
#Check some configurations
hexaConfig = await hexa.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexaConfig.pivotX, hexaConfig.pivotY, hexaConfig.pivotZ))
print("maxXY = ", hexaConfig.maxXY, "microns, maxZ= ", hexaConfig.maxZ, " microns")
print("maxUV = ", hexaConfig.maxUV, "deg, maxW= ", hexaConfig.maxW, " deg")

In [ ]:
settings = hexa.evt_settingsApplied.get()
has_settings = 0
if hasattr(settings, 'settingsVersion'):
    print(settings.settingsVersion)
    has_settings = 1
if (not has_settings) or (not settings.settingsVersion[:12] == 'default.yaml'):
    print('YOU NEED TO SEND THIS HEXAPOD TO STANDBY, THEN LOAD THE PROPER CONFIG')

In [ ]:
#only print the first 10 messages; if more than 10, ignore the rest
class MyLogHandler:
    def __init__(self, nmsg):
        self.nmsg = nmsg
        self.nprint = 0
    def printLogMessage(self, data):
        if self.nprint < self.nmsg:
            print(f"{data.level}: {data.message}")
        self.nprint += 1
my_log10 = MyLogHandler(10)
hexa.evt_logMessage.callback = my_log10.printLogMessage
await hexa.cmd_setLogLevel.set_start(level=10, timeout=5)

### Temperature

In [ ]:
ess = salobj.Remote(script.domain, "ESS")

In [ ]:
await ess.start_task

In [ ]:
temp = await ess.tel_temperature8Ch.next(flush=True, timeout=5)

In [ ]:
print([getattr(temp,'temperature%2d'%i) for i in range(8)])

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)
df = await client.select_time_series('lsst.sal.ESS.temperature8Ch', '*', start, end, csc_index)
fig, ax = plt.subplots(1,1, figsize=(15,4))
for i in range(8):
    plt.plot(getattr(df, 'temperatureC%02d'%i)
plt.grid()

### test state transition. If hexa is already enabled, disable then enable it.

In [ ]:
state = await hexa.evt_summaryState.aget(timeout=5)
print('starting with: hexa state', salobj.State(state.summaryState), pd.to_datetime(state.private_sndStamp, unit='s'))
dstate = await hexa.evt_controllerState.aget(timeout=5)
print('starting with: hexa state', MTHexapod.EnabledSubstate(dstate.enabledSubstate), 
      pd.to_datetime(dstate.private_sndStamp, unit='s'))

In [ ]:
#the below requires setting the hexapod to commandablebyDDS using the EUI
await salobj.set_summary_state(remote=hexa, state=salobj.State.STANDBY)

In [ ]:
#enable hexa,and make sure we use "default" settings
await salobj.set_summary_state(hexa, salobj.State.ENABLED, settingsToApply="default") 

In [ ]:
lut_mode = await hexa.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lut_mode.enabled)

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)
df = await client.select_time_series('lsst.sal.MTHexapod.logevent_controllerState', '*', start, end, csc_index)
if len(df) == 0:
    print('no record')
else:
    idx=df.MTHexapodID==1
    df = df[idx]
    fig, ax = plt.subplots(1,1, figsize=(15,4))
    plt.plot(df.enabledSubstate, '-o')
    plt.grid()

In [ ]:
df

In [ ]:
while True: #may need to wait a few seconds before event shows up in EFD
    end = Time(datetime.now(), scale='tai')
    start = end - timedelta(hours=1)
    dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start, end, csc_index)
    if len(dfe)>0:
        break

In [ ]:
dfe

### check the behavior when LUT is disabled.

In [ ]:
lut_mode = await hexa.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lut_mode.enabled)

In [ ]:
async def printHexaPosition(hexa):
    pos = await hexa.tel_application.next(flush=True, timeout=10.)
    print("Current Hexapod position")
    print(" ".join(f"{p:10.2f}" for p in pos.position[:3]), end = ' ') 
    print(" ".join(f"{p:10.6f}" for p in pos.position[3:]) )
await printHexaPosition(hexa)

If you want to observe the motions in chronograf, consider using "AND MTHexapodID={hexaId}" to filter out telemetry from the other hexapod

In [ ]:
async def moveHexaTo0(hexa):
    ### command it to collimated position (based on LUT)
    hexa.evt_inPosition.flush()
    #according to XML, units are micron and degree
    await hexa.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)
    while True:
        state = await hexa.evt_inPosition.next(flush=False, timeout=10)
        print("hexa in position?",state.inPosition, pd.to_datetime(state.private_sndStamp, unit='s'))
        if state.inPosition:
            break
    await printHexaPosition(hexa)

In [ ]:
await moveHexaTo0(hexa)

In [ ]:
hexa.evt_inPosition.flush()
for step in range(5,-1,-1):
    await hexa.cmd_move.set_start(x=0,y=0,z=-50*step, u=0,v=0,w=0,sync=True)
    while True:
        state = await hexa.evt_inPosition.next(flush=False, timeout=10)
        print("hexa in position?",state.inPosition, pd.to_datetime(state.private_sndStamp, unit='s'))
        if state.inPosition:
            break
    await printHexaPosition(hexa)

In [ ]:
hexa.evt_inPosition.flush()
for step in [1,2,3,-3,-2,-1]:
    #according to XML, units are micron and degree
    await hexa.cmd_offset.set_start(x=0,y=0,z=50*step, u=0,v=0,w=0,sync=True)
    while True:
        state = await hexa.evt_inPosition.next(flush=False, timeout=10)
        print("hexa in position?",state.inPosition, pd.to_datetime(state.private_sndStamp, unit='s'))
        if state.inPosition:
            break
    await printHexaPosition(hexa)
end = Time(datetime.now(), scale='tai')

In [ ]:
#end = Time('2021-04-26T17:08:30', scale = 'tai')
end = Time(datetime.now(), scale='tai')
end = end+timedelta(seconds=37)
start = end - timedelta(seconds=160)
df = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end, csc_index)
idx=df.MTHexapodID==1
df = df[idx]
print(end)
print(datetime.now())

dfip = await client.select_time_series('lsst.sal.MTHexapod.logevent_inPosition', '*', start, end, csc_index)
idx=dfip.MTHexapodID==1
dfip = dfip[idx]

fig, ax = plt.subplots(figsize=(19,3))
plt.plot(df.position2, label='zPosition')
plt.plot(dfip.inPosition*100, label='inPosition (1 or 0) x 100')
plt.grid()
plt.legend()

In [ ]:
len(dfip)

### Before we enable the LUT, use the mount controller to publish 

* mount target elevation 
* mount actual elevation 

The hexapod LUT needs the target elevation, target azimuth, temperature, and rotator angle. The only thing matters right now is the target elevation (everything else is set to zero). Check evt_compensationOffset to make sure the LUT is being calculated using the proper inputs.

### When the LUT is enabled

In [ ]:
await hexa.cmd_setCompensationMode.set_start(enable=1, timeout=10)

In [ ]:
lut_mode = await hexa.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lut_mode.enabled, pd.to_datetime(lut_mode.private_sndStamp, unit='s'))

In [ ]:
end = Time(datetime.now()+timedelta(seconds=37), scale='tai')
start = end - timedelta(seconds=100)
df = await client.select_time_series('lsst.sal.MTHexapod.logevent_controllerState', '*', start, end, csc_index)
if len(df) == 0:
    print('no record')
else:
    idx=df.MTHexapodID==1
    df = df[idx]
    fig, ax = plt.subplots(1,1, figsize=(15,4))
    plt.plot(df.enabledSubstate, '-o')
    plt.grid()
#as of 4/27/21, this shows the enabledSubstate oscillating between Stationary and MovingPointToPoint.

In [ ]:
async def printHexaUncompensatedAndCompensated(hexa):
    posU = await hexa.evt_uncompensatedPosition.aget(timeout=10.)
    print('Uncompensated position')
    print(" ".join(f"{p:10.2f}" for p in [getattr(posU, i) for i in 'xyz']), end = '    ')
    print(" ".join(f"{p:10.6f}" for p in [getattr(posU, i) for i in 'uvw']),'  ',
         pd.to_datetime(posU.private_sndStamp, unit='s'))    
    posC = await hexa.evt_compensatedPosition.aget(timeout=10.)
    print('Compensated position')
    print(" ".join(f"{p:10.2f}" for p in [getattr(posC, i) for i in 'xyz']), end = '     ')
    print(" ".join(f"{p:10.6f}" for p in [getattr(posC, i) for i in 'uvw']),'  ',
         pd.to_datetime(posC.private_sndStamp, unit='s'))

In [ ]:
await printHexaPosition(hexa)
await printHexaUncompensatedAndCompensated(hexa)

The inputs to the LUT are currently -
* elevation (from mount telemetry) 
* temperature (mount truss? not implemented yet)
* azimuth (not implemented)
* rotator angle (not implemented)

In [ ]:
a = hexa.evt_compensationOffset.get()
print(a.elevation, a.azimuth, a.rotation, a.temperature, pd.to_datetime(a.private_sndStamp, unit='s'))
print('x,y,z,u,v,w = ', a.x, a.y, a.z, a.u, a.v, a.w)

In [ ]:
mount_angle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mount_angle.actualPosition)
elev = mount_angle.actualPosition

### 1st way to change elevation: command the simulated mount (may need mount controller to get the evt_target() )

In [ ]:
await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=75)

In [ ]:
mount.cmd_trackTarget.set_start(azimuth=0, elevation=82) 
#Russell: this should output evt_target; 
#Tiago: may need to set up a constant stream of trackTargets

In [ ]:
print(mount.evt_target.get())

In [ ]:
mount_angle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mount_angle.actualPosition)
elev = mount_angle.actualPosition

In [ ]:
mount.cmd_stopTracking.set_start()

In [ ]:
LUTfile = '%s/notebooks/ts_config_mttcs/MTHexapod/v1/default.yaml'%(os.environ["HOME"])
with open(LUTfile, 'r') as stream:
    aa = yaml.safe_load(stream)
if hexa.salinfo.index == 1:
    elevCoeff = aa['camera_config']['elevation_coeffs']
    tCoeff = aa['camera_config']['temperature_coeffs']
elif hexa.salinfo.index == 2:
    elevCoeff = aa['m2_config']['elevation_coeffs']
    tCoeff = aa['m2_config']['temperature_coeffs']

In [ ]:
a = hexa.evt_compensationOffset.get()
print(a.elevation, a.azimuth, a.rotation, a.temperature, pd.to_datetime(a.private_sndStamp, unit='s'))
print('x,y,z,u,v,w = ', a.x, a.y, a.z, a.u, a.v, a.w)

In [ ]:
mt_target = await mount.evt_target.next(flush=False, timeout=10.)

In [ ]:
print(mount.evt_target.get())

In [ ]:
elev = a.elevation
async def printPredictedComp(elevCoeff, elev):
    '''
    This function deals with the elevation component of the LUT only, for now.
    We will add temperature, azimuth, and rotator angle when they are implemented.
    '''
    pred = []
    print('Predicted LUT compensation:')
    for i in range(6):
        coeff = elevCoeff[i] #starts with C0
        mypoly = np.polynomial.Polynomial(coeff)
        #mypoly = np.poly1d(coeff[::-1]) #if you use poly1d, be aware: it needs C5 first
        pred.append(mypoly(elev))
    print(" ".join(f"{p:10.2f}" for p in pred))
await printHexaPosition(hexa)    
await printPredictedComp(elevCoeff, elev)
await printHexaUncompensatedAndCompensated(hexa)

In [ ]:
await moveHexaTo0(hexa)

In [ ]:
await printHexaPosition(hexa)    
await printPredictedComp(elevCoeff, elev)
await printHexaUncompensatedAndCompensated(hexa)

### Now change the target (using mount controller), and move to the target (below, manually)

In [ ]:
await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=45)

In [ ]:
mount_angle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mount_angle.actualPosition)
elev = mount_angle.actualPosition

In [ ]:
await printHexaPosition(hexa)
await printPredictedComp(elevCoeff, elev)
await printHexaUncompensatedAndCompensated(hexa)

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=10)
df = await client.select_time_series('lsst.sal.MTHexapod.logevent_controllerState', '*', start, end, csc_index)
idx=df.MTHexapodID==1
df = df[idx]

fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(df.enabledSubstate, '-o')
plt.grid()

### 2nd way to change elevation: Do a slew, then check the LUT again

In [ ]:
if summit and use_ptg:
    #comment and uncomment the below depending on which components are expected to be on
    #by default, MTCS checks on every component (during the slew).
    mtcs.check.mtaos = False
    mtcs.check.mtm1m3 = False
    mtcs.check.mtm2 = False
    mtcs.check.mthexapod_1 = False
    mtcs.check.mthexapod_2 = False
    mtcs.check.mtdome = False
    mtcs.check.mtdometrajectory = False

In [ ]:
await salobj.set_summary_state(ptg, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
if use_ptg:
    alt = 34 * u.deg
    az = 0 * u.deg

    target_name="TMA motion test"
    time_data = await ptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_ptg = Time(time_data.mjd, format="mjd", scale="tai")
    time_err = curr_time_ptg - Time.now()
    print(f"Time error={time_err.sec:0.2f} sec")

    print(curr_time_ptg.tai.value)

    cmd_elaz = AltAz(alt=alt, az=az, 
                    obstime=curr_time_ptg.tai, 
                    location=mtcs.location)
    cmd_radec = cmd_elaz.transform_to(ICRS)
    await mtcs.slew_icrs(ra=cmd_radec.ra, dec=cmd_radec.dec, rot=0., rot_type=RotType.PhysicalSky)

In [ ]:
time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
ra = time_and_date.lst +0.5
dec = -34
print(ra)
await mtcs.slew_icrs(ra=ra, dec=dec, rot=0., rot_type=RotType.PhysicalSky)

In [ ]:
dec = -34.
for j in range(2):
    time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
    ra = time_and_date.lst + 0.5
    for i in range(2):
        print(ra, dec)
        await mtcs.slew_icrs(ra=ra, dec=dec, rot=0., rot_type=RotType.PhysicalSky)
        await asyncio.sleep(39.)
        ra -= 3.5/15.0
if use_ptg:
    await ptg.cmd_stopTracking.set_start(timeout=5.)

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
print('Are we tracking?', mountStatus.elevation , mountStatus.azimuth , rotStatus.inPosition)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)

In [ ]:
async def moveHexaTo100(hexa):
    ### command it to collimated position (based on LUT)
    hexa.evt_inPosition.flush()
    #according to XML, units are micron and degree
    await hexa.cmd_move.set_start(x=0,y=0,z=100, u=0,v=0,w=0,sync=True)
    while True:
        state = await hexa.evt_inPosition.next(flush=False, timeout=10)
        print("hexa in position?",state.inPosition, pd.to_datetime(state.private_sndStamp, unit='s'))
        if state.inPosition:
            break
    await printHexaPosition(hexa)

In [ ]:
await moveHexaTo100(hexa)

In [ ]:
await printHexaPosition(hexa)
await printPredictedComp(elevCoeff, elev)
await printHexaUncompensatedAndCompensated(hexa)

In [ ]:
state = await hexa.evt_summaryState.aget(timeout=5)
print('staring with: hexa state', salobj.State(state.summaryState), pd.to_datetime(state.private_sndStamp, unit='s'))
dstate = await hexa.evt_controllerState.aget(timeout=5)
print('staring with: hexa state', MTHexapod.EnabledSubstate(dstate.enabledSubstate), 
      pd.to_datetime(dstate.private_sndStamp, unit='s'))

### check angle and LUT after the slew

In [ ]:
mount_angle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mount_angle.angleActual)
elev = mount_angle.angleActual

In [ ]:
await printHexaPosition(hexa)
await printPredictedComp(elevCoeff, elev)
await printHexaUncompensatedAndCompensated(hexa)

### Check if the telescope is in tracking mode. If yes, need to stop stacking. 
The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

### When a new move command is issued before a previous move finishes, the desired behavior is to stop the first, reset a new target position, and move to the new target position. The below is used for testing this behavior.

In [ ]:
await hexa.cmd_setCompensationMode.set_start(enable=0, timeout=10)

In [ ]:
await moveHexaTo0(hexa)

In [ ]:
#await hexa.cmd_move.set_start(x=5660,y=0,z=7730,u=0.07,v=0,w=0,sync=True)
await hexa.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
await asyncio.sleep(.1)
await hexa.cmd_move.set_start(x=0,y=0,z=0,u=0,v=0,w=0,sync=True)
#await hexa.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
#await moveHexaTo0(hexa)
#await asyncio.sleep(39)

In [ ]:
await hexa.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
await hexa.cmd_stop.set_start()

In [ ]:
await hexa.cmd_move.set_start(x=0,y=0,z=0,u=0,v=0,w=0,sync=True)
await asyncio.sleep(.1)
await hexa.cmd_stop.set_start()

In [ ]:
await hexa.cmd_move.set_start(x=0,y=0,z=-50,u=0,v=0.0,w=0,sync=True)
await hexa.cmd_stop.set_start()

In [ ]:
#await hexa.cmd_move.set_start(x=0,y=0,z=-50,u=0,v=0.0,w=0,sync=True)
await hexa.cmd_move.set_start(x=0,y=0,z=-200,u=0,v=0.0,w=0,sync=True)

In [ ]:
end = Time(datetime.now(), scale='tai')
end = end+timedelta(seconds=37)
start = end - timedelta(seconds=160)
df = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end, csc_index)
idx=df.MTHexapodID==1
df = df[idx]
print(end)
print(datetime.now())

#dfip = await client.select_time_series('lsst.sal.MTHexapod.logevent_inPosition', '*', start, end, csc_index)
#idx=dfip.MTHexapodID==1
#dfip = dfip[idx]

fig, ax = plt.subplots(figsize=(19,3))
plt.plot(df.position2, label='zPosition')
#plt.plot(dfip.inPosition*100, label='inPosition (1 or 0) x 100')
plt.grid()
plt.legend()